In [27]:
# Generic OpenAI-compatible endpoint using ChatOpenAI
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.messages import HumanMessage,SystemMessage
import os
default_model_name = os.environ["OPENAI_MODEL"]
pg_connection = os.environ["PGVECTOR_CONNECTION_STRING"]
llm = ChatOpenAI(model=default_model_name,temperature=0)

In [3]:
from langchain_ollama import OllamaEmbeddings 
from langchain_postgres.vectorstores import PGVector
 
 
# embed each chunk and insert it into the vector store
embeddings_model = OllamaEmbeddings(model="nomic-embed-text")
# Coba dengan nama default
try:
    db = PGVector.from_existing_index(
        embedding=embeddings_model,
        collection_name="langchain",  # Coba default
        connection=pg_connection,
    )
    print("Berhasil konek dengan collection 'langchain'")
except Exception as e1:
    print(f"Collection 'langchain' tidak ditemukan: {e1}")
    
    # Coba dengan nama lain yang mungkin
    try:
        db = PGVector.from_existing_index(
            embedding=embeddings_model,
            collection_name="nomic-embed-text",  # Nama model
            connection=pg_connection,
        )
        print("Berhasil konek dengan collection 'nomic-embed-text'")
    except Exception as e2:
        print(f"Collection 'nomic-embed-text' tidak ditemukan: {e2}")

Berhasil konek dengan collection 'langchain'


In [4]:
db.similarity_search("Q3", k=4)


[Document(id='1677ab38-7604-4515-869a-ddf06cf5bae8', metadata={'source': '../../tesla.md'}, page_content='---\n\n PART III\n\nITEM 10\\. DIRECTORS, EXECUTIVE OFFICERS AND CORPORATE GOVERNANCE\n\nThe information required by this Item 10 of Form 10-K will be included in our 2023 Proxy Statement to be filed with the Securities and Exchange Commission in connection with the solicitation of proxies for our 2023 Annual Meeting of Stockholders and is incorporated herein by reference. The 2023 Proxy Statement will be filed with the Securities and Exchange Commission within 120 days after the end of the fiscal year to which this report relates. \n\nITEM 11\\. EXECUTIVE COMPENSATION\n\nThe information required by this Item 11 of Form 10-K will be included in our 2023 Proxy Statement and is incorporated herein by reference. \n\nITEM 12\\. SECURITY OWNERSHIP OF CERTAIN BENEFICIAL OWNERS AND MANAGEMENT AND RELATED STOCKHOLDER MATTERS\n\nThe information required by this Item 12 of Form 10-K will be 

In [37]:
# create retriever
# retriever = db.as_retriever(search_kwargs={"k": 2})

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import chain






retriever = db.as_retriever(search_kwargs={"k": 2})

prompt = ChatPromptTemplate.from_template("""Always answer in indonesian language.Answer the question based only on
    the following context:
{context}

Question: {question}
""")
chain = prompt | llm
rewrite_prompt = ChatPromptTemplate.from_template("""Provide a better search
    query for web search engine to answer the given question, end the queries
    with '**'.
    examples:
    Question: Minyak
    Answer: Jenis Minyak**
    Question: {x} Answer:""")

def parse_rewriter_output(message):
    return message.content.strip('"').strip("**")

rewriter = rewrite_prompt | llm | parse_rewriter_output

 
def qa_rrr(input):
    # rewrite the query
    new_query = rewriter.invoke(input)
    print(f"q:{new_query}")
    # fetch relevant documents
    docs = retriever.get_relevant_documents(new_query)
    # format prompt
    formatted = prompt.invoke({"context": docs, "question": input})
    # generate answer
    answer = llm.invoke(formatted)
    return answer
 
result=qa_rrr("Apa saja produk TESLA?")
print(result.content)

q:Daftar Produk Tesla lengkap
Berdasarkan dokumen yang tersedia, produk Tesla meliputi:

1. **Kendaraan listrik (mobil)** - Termasuk kendaraan baru dan bekas yang dijual oleh Tesla, dilengkapi dengan jaminan terbatas pabrikan.
2. **Sistem penyimpanan energi** - Produk yang berkaitan dengan pembangkitan dan penyimpanan energi, seperti baterai rumah atau sistem penyimpanan energi lainnya.
3. **Layanan perluasan jaminan dan rencana layanan tambahan** - Untuk kendaraan dan produk energi tertentu, Tesla menawarkan paket layanan tambahan yang dapat dibeli untuk memperpanjang cakupan garansi.

Selain itu, Tesla juga menyediakan layanan perbaikan dan perawatan untuk produk-produk tersebut, termasuk layanan instalasi untuk sistem energi, lengkap dengan jaminan terhadap pekerjaan instalasi tersebut.


In [40]:
from langchain.prompts import ChatPromptTemplate
perspectives_prompt = ChatPromptTemplate.from_template(
    """You are an AI language model assistant. Your task is to generate five different versions 
    of the given user question to retrieve relevant documents from a vector database.
    By generating multiple perspectives on the user question, your goal is to help the user 
    overcome some of the limitations of the distance-based similarity search. Provide these 
    alternative questions separated by newlines. Original question: {question}"""
)
def parse_queries_output(message):
    return message.content.split('\n')

query_gen = perspectives_prompt | llm | parse_queries_output

def get_unique_union(document_lists):
    # Flatten list of lists, and dedupe them
    deduped_docs = {
        doc.page_content: doc
        for sublist in document_lists for doc in sublist
    }
    # return a flat list of unique docs
    return list(deduped_docs.values())

retrieval_chain = query_gen | retriever.batch | get_unique_union

def multi_query_qa(input):
    # fetch relevant documents
    docs = retrieval_chain.invoke(input)
    # format prompt
    formatted = prompt.invoke({"context": docs, "question": input})
    # generate answer
    answer = llm.invoke(formatted)
    return answer

result=multi_query_qa("cabe hujau?")
print(result.content)

Cabai hijau adalah jenis cabai yang dipanen pada tahap kematangan awal, sebelum terjadi perubahan warna. Berdasarkan informasi dari dokumen, cabai hijau biasanya dipanen sekitar 70-80 hari setelah tanam. Pemanenan dilakukan dengan cara memotong tangkai buah menggunakan gunting atau pisau tajam, dan sebaiknya dilakukan di pagi hari setelah embun kering atau di sore hari untuk mengurangi stres pada tanaman.
